# RNN을 이용하여 텍스트 생성하기

1. 데이터에 대한 이해와 전처리

1-1. 라이브러리 불러오기

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import numpy as np 
from tensorflow.keras.utils import to_categorical 

C:\Users\isfs0\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\isfs0\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\isfs0\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


코드설명 - 라이브러리 : 주기능

1. tensorflow.keras.preprocessing.text.Tokenizer:

텍스트 데이터를 토큰화하고 단어를 정수로 매핑하는 데 사용됩니다.
fit_on_texts(): 주어진 텍스트 데이터에 대해 단어 사전을 만듭니다.
texts_to_sequences(): 텍스트 데이터를 정수 시퀀스로 변환합니다.
word_index: 단어와 해당 정수 인덱스 사전을 확인할 수 있습니다.

2. tensorflow.keras.preprocessing.sequence.pad_sequences:

시퀀스 데이터의 패딩(padding)을 조절하는 데 사용됩니다.
시퀀스의 길이를 맞추기 위해 0 또는 다른 값으로 시퀀스를 채웁니다.
패딩된 시퀀스를 모델에 입력으로 사용할 수 있도록 합니다.

3. numpy:

다차원 배열과 행렬 연산을 수행하는 파이썬 라이브러리입니다.
주로 데이터 배열을 다루는 데 사용됩니다.

4. tensorflow.keras.utils.to_categorical:

다중 클래스 분류(Classification) 문제에서 타겟(레이블)을 원-핫 인코딩 형식으로 변환하는 데 사용됩니다.
각 클래스에 대한 이진 벡터로 변환하여 모델이 클래스 확률을 출력하기 쉽게 만듭니다.

1-2. 임의의 텍스트 데이터 생성

In [2]:
# 텍스트 주어주기
text="""경마장에  있는  말이  뛰고  있다\n 그의  말이  법이다\n 가는  말이  고와야  오는  말이  곱다\n"""     

1-3. Tokenizer와 단어집합 사이즈 정해주기 

In [3]:
t = Tokenizer() 
# Tokenizer 객체 t 생성, 텍스트 데이터 'text'를 토큰화하고
# 단어를 정수로 매핑하는데에 사용

t.fit_on_texts([text]) # Mapping 진행 
# t를 사용하여 주어진 텍스트 데이터(text)를 토큰화하고 단어 집합을 생성합니다. 
# 여기서 text는 리스트 형태로 주어진 텍스트 데이터입니다.

vocab_size = len(t.word_index) + 1 
# 단어 집합의 크기를 계산
# word_index 속성에는 단어와 해당 정수 인덱스의 매핑이 저장되어 있음
# 따라서 이 속성의 길이에 1을 더하여 전체 단어 집합의 크기를 계산
# * 1을 더하는 이유 : 
# 정수 인덱스가 1부터 시작하고 0은 보통 패딩용으로 예약되기 때문

print('단어 집합의  크기  : %d' % vocab_size)
# 계산된 단어 집합의 크기를 출력합니다. 
# 이 크기는 텍스트 데이터에서 고유한 단어의 수를 나타냅

단어 집합의  크기  : 12


1-4. t.word_index의 값 확인하기(집합의 크기는 12이므로, 집합의 요소들을 확인)

In [4]:
print(t.word_index) 
# 각  단어와  단어에  부여된  정수  인덱스  출력 : ** 빈도수 순으로 출력
# index를 사용하지만 0은 사용하지 않음 따라서 1~11 로 출력

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


'말이' 라는 단어가 출력되는 빈도가 가장 높으므로 인덱스값이 1로 설정

In [ ]:
1-5. sequence를 저장

In [5]:
sequences = list()
# 생성한 시퀀스를 저장할 리스트

for line in text.split('\n'):  
    # 줄 바꿈(Wn)을 기준으로 문장을 분리
    
    print(t.texts_to_sequences([line]))  
    # 현재 문장을 정수 시퀀스로 변환하여 출력
    encoded = t.texts_to_sequences([line])[0]  
    # texts_to_sequences 결과의 0번째 위치에 문장의 정수 시퀀스가 있음

    # 문장의 일부를 사용하여 시퀀스 생성
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)  # 생성한 시퀀스를 리스트에 추가

print('학습에 사용한 샘플의 개수: %d' % len(sequences))  # 생성된 학습 샘플의 개수 출력

[[2, 3, 1, 4, 5]]
[[6, 1, 7]]
[[8, 1, 9, 10, 1, 11]]
[[]]
학습에 사용한 샘플의 개수: 11


sequences = list(): 생성한 시퀀스(문장의 일부와 레이블)를 저장할 빈 리스트를 초기화합니다.

for line in text.split('\n'):: 입력 텍스트(text)를 줄 바꿈 문자(\n)를 기준으로 문장으로 분리합니다.

encoded = t.texts_to_sequences([line])[0]: 현재 문장을 Tokenizer 객체 t를 사용하여 정수 시퀀스로 변환하고, 이 정수 시퀀스를 encoded에 저장합니다.

for i in range(1, len(encoded)):: 현재 문장의 정수 시퀀스를 이용하여 학습 샘플을 생성합니다. 이때, i를 1부터 encoded의 길이까지 변화시키면서, 각각의 시퀀스를 생성합니다.

sequences.append(sequence): 생성한 시퀀스를 sequences 리스트에 추가합니다. 이 시퀀스는 문장의 일부와 그 다음에 오는 단어(레이블)로 구성됩니다.

print('학습에 사용한 샘플의 개수: %d' % len(sequences)): 생성된 학습 샘플의 개수를 출력합니다. 이것은 학습에 사용할 수 있는 시퀀스 데이터의 총 개수를 나타냅니다.

In [6]:
print(sequences) # 전체  샘플을  출력

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [7]:
max_len=max(len(l) for l in sequences) # 모든  샘플에서  길이가  가장  긴  샘플의  길이  출력 
print('샘플의  최대  길이  : {}'.format(max_len))

샘플의  최대  길이  : 6


1-5. 패딩 설정하기

In [8]:
# 전체체  샘플의  길이를  6으로  패딩 
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre') 
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


1-6. 넘파이배열로 시퀀스를 저장, 학습과 정답데이터 쥐어주기

In [9]:
sequences = np.array(sequences) 

X = sequences[:,:-1] # 학습  데이터 
y = sequences[:,-1]  # 정답(Label) 데이터 print(X) 

print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


1-7. 원핫인코딩

In [10]:
y = to_categorical(y, num_classes=vocab_size) # 원-핫  인코딩  수행 
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


#### 2. 모델설계하기


2-1. 라이브러리 불러오기

In [11]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

#### What is the Embedding?
#### -> 단어를 고정된 차원의 벡터로 표현하는 기술

`tf.keras.layers.Embedding( input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None, **kwargs)`

- 인수
    - input_dim : 입력 크기
    - output_dim : 출력 크기
    - input_length : 입력 데이터의 길이

단어를 밀집벡터로 만드는 일을 수행한다. 정수 인코딩이 된 단어들을 입력으로 받아 수행한다.

2-2. 모델설정과 구성

In [13]:
model = Sequential() 
model.add(Embedding(vocab_size, 6, input_length=max_len-1)) 
model.add(SimpleRNN(32))  # SimpleRNN 오류발생하면 numpy 버전을 낮춤( pip install -U numpy==1.19.5) 
model.add(Dense(vocab_size, activation='softmax')) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 6)              72        
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1248      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1716 (6.70 KB)
Trainable params: 1716 (6.70 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


'model = Sequential()'
- Sequential()을 사용하여 Sequential 모델을 생성합니다. 
- Sequential 모델은 레이어를 순차적으로 쌓는 데 사용됩니다.

'model.add(Embedding(vocab_size, 6, input_length=max_len-1)) '
- Embedding 레이어를 추가합니다. 
- 이 레이어는 단어 임베딩을 수행하는데 사용됩니다. 
- vocab_size는 어휘 사전의 크기를 나타내며, 6은 임베딩 차원의 크기를 나타냅니다. input_length는 입력 시퀀스의 길이를 나타냅니다.

'model.add(SimpleRNN(32))'
- SimpleRNN 레이어를 추가합니다. 
- 이 레이어는 32개의 유닛을 가지는 Simple RNN 레이어입니다.

'model.add(Dense(vocab_size, activation='softmax'))' 
- Dense 레이어를 추가합니다. 
- 출력 레이어로 사용되며, vocab_size만큼의 뉴런을 가지며 
- 활성화 함수로 소프트맥스(softmax)를 사용합니다. 
- 이것은 다중 클래스 분류 문제를 다루는데 적합합니다.

2-3. 모델컴파일 및 학습

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.5054 - accuracy: 0.0909 - 1s/epoch - 1s/step
Epoch 2/200
1/1 - 0s - loss: 2.4957 - accuracy: 0.1818 - 4ms/epoch - 4ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4863 - accuracy: 0.1818 - 3ms/epoch - 3ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4771 - accuracy: 0.0909 - 4ms/epoch - 4ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4680 - accuracy: 0.1818 - 0s/epoch - 0s/step
Epoch 6/200
1/1 - 0s - loss: 2.4589 - accuracy: 0.1818 - 0s/epoch - 0s/step
Epoch 7/200
1/1 - 0s - loss: 2.4497 - accuracy: 0.2727 - 5ms/epoch - 5ms/step
Epoch 8/200
1/1 - 0s - loss: 2.4402 - accuracy: 0.2727 - 5ms/epoch - 5ms/step
Epoch 9/200
1/1 - 0s - loss: 2.4304 - accuracy: 0.2727 - 4ms/epoch - 4ms/step
Epoch 10/200
1/1 - 0s - loss: 2.4202 - accuracy: 0.2727 - 0s/epoch - 0s/step
Epoch 11/200
1/1 - 0s - loss: 2.4094 - accuracy: 0.2727 - 0s/epoch - 0s/step
Epoch 12/200
1/1 - 0s - loss: 2.3980 - accuracy: 0.3636 - 17ms/epoch - 17ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3860 - accuracy: 0.3636 - 0s/epoch - 

Epoch 106/200
1/1 - 0s - loss: 0.7353 - accuracy: 0.8182 - 14ms/epoch - 14ms/step
Epoch 107/200
1/1 - 0s - loss: 0.7234 - accuracy: 0.8182 - 0s/epoch - 0s/step
Epoch 108/200
1/1 - 0s - loss: 0.7117 - accuracy: 0.8182 - 13ms/epoch - 13ms/step
Epoch 109/200
1/1 - 0s - loss: 0.7003 - accuracy: 0.8182 - 0s/epoch - 0s/step
Epoch 110/200
1/1 - 0s - loss: 0.6890 - accuracy: 0.8182 - 13ms/epoch - 13ms/step
Epoch 111/200
1/1 - 0s - loss: 0.6780 - accuracy: 0.8182 - 0s/epoch - 0s/step
Epoch 112/200
1/1 - 0s - loss: 0.6672 - accuracy: 0.8182 - 0s/epoch - 0s/step
Epoch 113/200
1/1 - 0s - loss: 0.6566 - accuracy: 0.8182 - 17ms/epoch - 17ms/step
Epoch 114/200
1/1 - 0s - loss: 0.6461 - accuracy: 0.8182 - 0s/epoch - 0s/step
Epoch 115/200
1/1 - 0s - loss: 0.6358 - accuracy: 0.8182 - 0s/epoch - 0s/step
Epoch 116/200
1/1 - 0s - loss: 0.6256 - accuracy: 0.8182 - 599us/epoch - 599us/step
Epoch 117/200
1/1 - 0s - loss: 0.6156 - accuracy: 0.8182 - 0s/epoch - 0s/step
Epoch 118/200
1/1 - 0s - loss: 0.6058 - ac

#### RNN의 알고리즘방식 간접체험하기

In [15]:
import numpy as np

def sentence_generation(model, t, current_word, n):
    init_word = current_word  # 입력된 시작 단어를 저장
    sentence = ''  # 생성된 문장을 저장할 변수

    for _ in range(n):  # n번 반복하면서 단어를 예측하고 문장을 생성
        encoded = t.texts_to_sequences([current_word])[0]  # 현재 단어에 대한 정수 인코딩을 수행
        print("encoded => ", encoded)
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')  # 데이터에 패딩을 추가하여 입력 시퀀스 길이를 맞춤
        result = model.predict(encoded, verbose=0)  # 모델을 사용하여 다음 단어를 예측
        result = np.argmax(result, axis=-1)  # 가장 확률이 높은 단어의 인덱스를 선택
        print("result => ", result)
        
        for word, index in t.word_index.items():  # 예측 결과를 단어로 변환
            if index == result:
                break
        current_word = current_word + ' '  + word  # 현재 단어와 예측한 단어를 이어붙임
        sentence = sentence + ' ' + word  # 예측한 단어를 문장에 추가

    sentence = init_word + sentence  # 초기 단어와 생성된 문장을 합쳐 최종 문장 생성
    return sentence

# 함수를 호출하여 주어진 모델과 입력 단어 "경마장에"를 이용하여 4개의 단어를 생성하고 출력
print(sentence_generation(model, t, '경마장에', 4))

encoded =>  [2]
result =>  [3]
encoded =>  [2, 3]
result =>  [1]
encoded =>  [2, 3, 1]
result =>  [4]
encoded =>  [2, 3, 1, 4]
result =>  [5]
경마장에 있는 말이 뛰고 있다


'model'
- 텍스트 생성에 사용되는 Keras 모델입니다.

't'
- 토크나이저 객체로, 단어와 정수 인덱스 간의 매핑을 가지고 있습니다.

'current_word' 
- 시작 단어로, 문장 생성이 시작되는 단어입니다.

'n' 
- 생성할 단어의 수를 나타내는 변수입니다.

'encoded'
- 현재 단어를 정수 시퀀스로 변환한 결과를 저장합니다.

'pad_sequences'
- 입력 시퀀스의 길이를 맞추기 위해 패딩을 추가합니다.

'result'
- 모델을 사용하여 다음 단어를 예측한 결과를 저장합니다.

't.word_index'
- 토크나이저에서 단어와 인덱스 간의 매핑을 확인하기 위해 사용됩니다.

In [16]:
print(sentence_generation(model, t, '경마장에', 4))  # 경마장에 다음에 나올 단어를 4개

encoded =>  [2]
result =>  [3]
encoded =>  [2, 3]
result =>  [1]
encoded =>  [2, 3, 1]
result =>  [4]
encoded =>  [2, 3, 1, 4]
result =>  [5]
경마장에 있는 말이 뛰고 있다


In [17]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측 
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

encoded =>  [6]
result =>  [1]
encoded =>  [6, 1]
result =>  [7]
그의 말이 법이다
encoded =>  [8]
result =>  [1]
encoded =>  [8, 1]
result =>  [9]
encoded =>  [8, 1, 9]
result =>  [10]
encoded =>  [8, 1, 9, 10]
result =>  [1]
encoded =>  [8, 1, 9, 10, 1]
result =>  [11]
가는 말이 고와야 오는 말이 곱다
